In [1]:
library(stats)

In [2]:
library(wendy)

In [3]:
solveWendy

function (f, p0, U, tt, lip = FALSE, noise_dist = c("addgaussian", 
    "lognormal"), method = c("MLE", "IRLS", "OLS"), control = NULL) 
{
    noise_dist <- match.arg(noise_dist)
    method <- match.arg(method)
    default_control <- list(optimize = TRUE, compute_svd = TRUE, 
        diag_reg = 1e-09, max_iterates = 200, S = 1, p = 16, 
        test_fun_type = "MSG", radius_params = 2^(0:3), radius_min_time = 0.1, 
        radius_max_time = 5, k_max = 200, max_test_fun_condition_number = 10000, 
        min_test_fun_info_number = 0.95)
    if (!is.null(control)) {
        control <- modifyList(default_control, control)
    }
    else {
        control <- default_control
    }
    if (noise_dist == "lognormal") {
        data <- preprocess_data(U, tt)
        U <- data$U
        tt <- data$tt
    }
    torch::torch_set_default_dtype(torch::torch_float64())
    sig <- torch::torch_tensor(estimate_std(U, k = 6), dtype = torch::torch_float64())
    test_fun_matrices <- if (control$test_fun_type == "SSL") {
        build_full_test_function_matrices_ssl(U, tt, control)
    }
    else {
        build_full_test_function_matrices_msg(U, tt, control, 
            control$compute_svd)
    }
    V <- torch::torch_tensor(test_fun_matrices$V, dtype = torch::torch_float64())
    Vp <- torch::torch_tensor(test_fun_matrices$V_prime, dtype = torch::torch_float64())
    min_radius <- test_fun_matrices$min_radius
    J <- length(p0)
    D <- ncol(U)
    mp1 <- nrow(U)
    K <- nrow(V)
    u_expr <- do.call(c, lapply(1:ncol(U), function(i) symengine::S(paste0("u", 
        i))))
    p_expr <- do.call(c, lapply(1:length(p0), function(i) symengine::S(paste0("p", 
        i))))
    t_expr <- symengine::S("t")
    f_expr <- switch(noise_dist, lognormal = lognormal_transform(f(u_expr, 
        p_expr, t_expr)), f(u_expr, p_expr, t_expr))
    J_u_sym <- compute_symbolic_jacobian(f_expr, u_expr)
    J_up_sym <- compute_symbolic_jacobian(J_u_sym, p_expr)
    J_p_sym <- compute_symbolic_jacobian(f_expr, p_expr)
    J_pp_sym <- compute_symbolic_jacobian(J_p_sym, p_expr)
    J_upp_sym <- compute_symbolic_jacobian(J_up_sym, p_expr)
    vars <- c(p_expr, u_expr, t_expr)
    f_ <- build_fn(f_expr, vars)
    J_u <- build_fn(J_u_sym, vars)
    J_up <- build_fn(J_up_sym, vars)
    J_p <- build_fn(J_p_sym, vars)
    J_pp <- build_fn(J_pp_sym, vars)
    J_upp <- build_fn(J_upp_sym, vars)
    F_ <- build_F(U, tt, f_, J)
    G <- build_G_matrix(V, U, tt, F_, J)
    g0 <- torch::torch_mm(V, F_(rep(0, J)))$reshape(c(-1))
    g <- ifelse(!lip, build_g(V, F_), build_g_linear(G))
    b <- -1 * torch::torch_mm(Vp, torch::torch_tensor(U, dtype = torch::torch_float64()))$reshape(c(-1))
    b <- if (!lip) 
        b
    else b - g0
    Jp_r <- ifelse(!lip, build_Jp_r(J_p, K, D, J, mp1, V, U, 
        tt), build_Jp_r_linear(G))
    Hp_r <- build_Hp_r(J_pp, K, D, J, mp1, V, U, tt)
    L0 <- build_L0(K, D, mp1, Vp, sig)
    L <- ifelse(!lip, build_L(U, tt, J_u, K, V, L0, sig, J), 
        build_L_linear(U, tt, J_u, K, V, L0, sig, J))
    Jp_L <- ifelse(!lip, build_Jp_L(U, tt, J_up, K, J, D, V, 
        sig), build_Jp_L_linear(U, tt, J_u, K, V, L0, sig, J))
    Hp_L <- build_Hp_L(U, tt, J_upp, K, J, D, V, sig)
    S <- build_S(L, diag_reg = control$diag_reg)
    Jp_S <- build_J_S(L, Jp_L, J, K, D)
    wnll <- build_wnll(S, g, b, K, D)
    J_wnll <- build_J_wnll(S, Jp_S, Jp_r, g, b, J)
    H_wnll <- ifelse(!lip, build_H_wnll(S, Jp_S, L, Jp_L, Hp_L, 
        Jp_r, Hp_r, g, b, J), build_H_wnll_linear(S, Jp_S, L, 
        Jp_L, Jp_r, g, b, J))
    res <- list()
    res$wnll <- wnll
    res$J_wnll <- J_wnll
    res$H_wnll <- H_wnll
    res$g <- g
    res$g0 <- g0
    res$G <- G
    res$b <- b
    res$f <- f_
    res$J_p <- J_p
    res$J_upp <- J_upp
    res$S <- S
    res$Jp_S <- Jp_S
    res$Jp_r <- Jp_r
    res$F_ <- F_
    res$f_sym <- f_expr
    res$L <- L
    res$sig <- sig
    res$V <- V
    res$V_prime <- Vp
    res$min_radius <- min_radius
    if (!control$optimize) 
        re